<div align="center">
    <img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Build%20AI%20agents%20and%20workflows%20with%20a%20simple%20API&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" width="640" height="320" />
</div>

## Task Definition: Designing Multi-Step Tasks using Brave Search Integration

### Overview



### Task Flow 

1. **Input**: 
   - User provides a topic for the three research questions to be based on

2. **Tool Call**:
   - Preforms a web search for each of the question

3. **Evaluate**:
   - Extract relevant information 

4. **Conditional Logic**:
   - Check to make sure that enough information has been gathered 

5. **Log**:
   - Records a summery

6. **Return**:
   - Returns the final output


## Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/DarkZelda11/julep/blob/dev/cookbooks/06-Designing_Multi-Step_Tasks.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:



### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>



Installing the Julep Client

In [155]:
!pip install julep -U --quiet

### Note:

- UUIDs are generated for both the agent and task to uniquely identify them within the system.
- Once created, these UUIDs should remain unchanged for simplicity.
- Altering a UUID will result in the system treating it as a new agent or task.
- If a UUID is changed, the original agent or task will continue to exist in the system alongside the new one.

In [156]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

## Creating Julep Client with the API Key 

In [157]:
from julep import Client

api_key = "" # Your Api key here

# Create a client
client = Client(api_key=api_key, environment='dev')

## Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent). 

In [158]:
# Defining the agent 
name = "Multi-Step Task Agent"
about = "An agent capable of executing complex multi-step tasks."
default_settings = {
    "temperature": 0.7,
    "top_p": 1,
    "min_p": 0.01,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "length_penalty": 1.0,
    "max_tokens": 150,
}

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

# Add a web search tool to the client
client.agents.tools.create(
    agent_id=AGENT_UUID,
    name="web_search",
    integration={
        "provider": "brave",
        "method": "search",
        "setup": {"api_key": ""},
    }, 
    type="integration"
)

ResourceCreated(id='9c343dd0-769f-49df-bb9b-e173bb457e1e', created_at=datetime.datetime(2024, 11, 18, 9, 36, 2, 872247, tzinfo=datetime.timezone.utc), jobs=[])

## Defining the task 

Tasks in Julep are GitHub Actions style workflows that define long-running, multistep actions.
You can use them to conduct complex actions by defining them step-by-step. They have access to all Julep integrations.

To learn more about tasks, visit [Julep documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#task).

In [159]:
import yaml

More on how to define a task can be found [here](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md).

In [160]:
task_def = yaml.safe_load(r"""
name: Multi-Step Task Demonstration

input_schema:
  type: object
  properties:
    topic:
      type: string
      description: The topic to research and summarize.

tools:
  - name: web_search
    type: integration
    integration:
      provider: brave
      setup:
        api_key: ""

main:
  # Step 1: Generate research questions
  - prompt:
      - role: system
        content: "You are a research assistant. Your task is to formulate three specific research questions about the given topic: {{inputs.topic}}"
    unwrap: true 

  # Step 2: Web search for each question
  - foreach:
      in: _.split('\\n').filter(line => line.trim() !== "" && /^[1-9]\./.test(line))
      do:
        tool: web_search
        arguments:
          query: line.trim().substring(3)

  # Step 3: Generate summary
  - prompt:
      - role: system
        content: |
          Based on the search results, provide a comprehensive summary about {{inputs.topic}}.
          Include key findings and insights from the research.
      - role: user
        content: "{{_}}"
    unwrap: true

  # Step 4: Return results
  - return: 
      summary: _,
      topic: inputs.topic
""")


Creating/Updating a task.

In [161]:
# Creating/Updating a task
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

## Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

Creates an execution workflow for the Task defined in the yaml file.

In [162]:
# Creating an Execution object
execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "topic": "Artificial Intelligence in Healthcare"
    }
)

In [163]:
import time
time.sleep(25)

In [164]:
print(f"Execution ID: {execution.id}")

Execution ID: b3a1625b-ee7c-4ef5-99f0-5f7fce99cd2c


In [165]:
# Get execution details
execution = client.executions.get(execution.id)
print("\nExecution Status:", execution.status)


Execution Status: running


Retrieves and lists all the steps of a defined task that have been executed up to that point in time. Unlike streaming, this function does not continuously monitor the execution; it only provides a snapshot of the steps completed so far without displaying real-time updates as the task progresses.

In [166]:
transitions = client.executions.transitions.list(execution_id=execution.id).items
print("Execution Steps:")
if execution.status == "running":
    for transition in transitions:
        print(transition.type)
        print(transition.output)
        print("_" * 80)

Execution Steps:
step
Topic: The Impact of Artificial Intelligence on Employment Trends

1. How has the adoption of artificial intelligence technologies in various industries affected employment rates and job displacement in different sectors over the past decade?

2. What skills and competencies are becoming increasingly important for workers to remain competitive in the job market as artificial intelligence continues to evolve and integrate into more industries?

3. In what ways are companies and governments addressing the potential ethical and socioeconomic challenges posed by AI-induced shifts in employment trends, and what strategies have proven most effective in mitigating negative impacts on workers?
________________________________________________________________________________
init
{'topic': 'Artificial Intelligence in Healthcare'}
________________________________________________________________________________


In [167]:
print(client.executions.transitions.list(execution.id))

SyncOffsetPagination[Transition](items=[Transition(id='237677f9-67b8-49a7-b8c5-31f1b83abc8d', created_at=datetime.datetime(2024, 11, 18, 9, 36, 29, 334210, tzinfo=datetime.timezone.utc), current=Current(step=0, workflow='main'), execution_id='b3a1625b-ee7c-4ef5-99f0-5f7fce99cd2c', next=Next(step=1, workflow='main'), output='Topic: The Impact of Artificial Intelligence on Employment Trends\n\n1. How has the adoption of artificial intelligence technologies in various industries affected employment rates and job displacement in different sectors over the past decade?\n\n2. What skills and competencies are becoming increasingly important for workers to remain competitive in the job market as artificial intelligence continues to evolve and integrate into more industries?\n\n3. In what ways are companies and governments addressing the potential ethical and socioeconomic challenges posed by AI-induced shifts in employment trends, and what strategies have proven most effective in mitigating ne

Continuously monitor and stream the steps of a defined task. It retrieves and displays the transitions or execution steps of the task identified by execution. id in real-time, showing each step sequentially until the task is either completed or an error causes it to terminate. 

In [168]:
# Stream execution transitions
print("\nStreaming execution transitions:")
for transition in client.executions.transitions.stream(execution_id=execution.id):
    try:
        # Since transition is a string, we need to print it directly
        print(f"Transition: {transition}")
        print("_" * 80)
    except Exception as e:
        print(f"Error processing transition: {e}")


Streaming execution transitions:
Transition: d
________________________________________________________________________________
Transition: a
________________________________________________________________________________
Transition: t
________________________________________________________________________________
Transition: a
________________________________________________________________________________
Transition: :
________________________________________________________________________________
Transition:  
________________________________________________________________________________
Transition: {
________________________________________________________________________________
Transition: '
________________________________________________________________________________
Transition: t
________________________________________________________________________________
Transition: r
________________________________________________________________________________
Transition: a
__